In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import lightgbm as lgb
import matplotlib
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
import warnings
from six.moves import urllib

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
id=test['ID_code']

In [ ]:
target_train = train['target'].values
train = train.drop(['target', 'ID_code'], axis=1)
test = test.drop(['ID_code'], axis=1)

In [ ]:
train_stats = train.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
train = norm(train)
test = norm(test)

In [ ]:
sc=StandardScaler()
train=sc.fit_transform(train)
test=sc.transform(test)

In [ ]:
xgb_preds = []
K = 8
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

In [ ]:
for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    xgb_params = {'max_depth': 6, 'objective': 'binary:logistic','eval_metric':'auc'}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train,200,watchlist,early_stopping_rounds=100)                     
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

In [ ]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

output = pd.DataFrame({'ID_code': id, 'target': preds})
output.to_csv("{}-foldCV_avg_sub.csv".format(K), index=False) 